# Web Services with Flask
*Due Tuesday, February 26, 5 PM*

    
In this Lab we explore the construction of a web service with [Flask](http://flask.pocoo.org), a microframework for Python. We chose Flask because it's a prominent framework for Python, it's simple, and it's emblematic of frameworks for in other languages. Learn Flask and you'll find NodeJS (Javascript), Rails or Sinatra (Ruby), or Beego (Golang) immediately familiar. That's because the concepts behind all web frameworks are essentially similar, although details vary considerably. 

## A University App

Over the next few weeks we're going to construct a fairly substantial web service and app, one that models a simplified version of a university. 

This university will consist of the following entities:

* `students` (because, of course)
* `professors` (an unnecessary evil)
* `courses` (hey, why not?)
* `assignments` (you hate them, we hate 'em too)
*  `grades` (the meaning of life)

In this simulated world, professors create and manage courses, students sign up for these courses, professors create assignents and grade them. These abstractions are linked together. For example, courses belong to professors and have students and assignments. 

The university app will have several components:

1. An API, implemented as a web service.
2. A persistence layer that models the entities above (students, courses, etc) in a relational database.
2. A model layer that implements the 'logic' of the application. The model sits between the API and the persistence layer. It abstracts and hides the details of persistence away from the API layer.
2. Clients. These can be web apps, desktop apps, mobile apps, or even command-line tools. What's important is that this architecture cleanly separates the UI from the API.

*Question*: you might be wondering about the connection between this exercise and data engineering. 

*Answer*? Absolutely nothing. At least not the surface. But, again, the concepts will translate. Someday you might be asked to create a data warehouse. That warehouse will be modeled very similarly to the one here.

## Concepts and Plan

The accompanying lecture covers the main concepts: web services, their relevance to data science, HTTP, REST APIs, routing, and so on. We'll assume you are familiar with the basics already.

The plan with the university app is to work incrementally: to begin on the outside and proceed inward. Here 'outside' is the API. We'll begin by 'stubbing' out one. That is, we'll create one resource (users--students and professors) and define the operations we might want to perform on these users. Over the next few weeks we'll add more resources (e.g., assignments) and create a persistence layer with a database. By the time you are done, you will have a complete app that can be deployed to the cloud and accessed from command-line clients, and web, mobile or desktop apps.

## Users: Students and Professors


For simplicity, we'll amalgamate students and professors into a single resource called `user` and delineate them by a `role` attribute. A single `user` resource will simplify authentication (which we'll skip in this app) and allow us to implement __authorization__ students from grading assignments and not preclude professors from taking courses.

A real app might track information about a professor separately (e.g., rank, salary, hire date) separately, and similarly for students. This information could be linked to the user depending on their role.

We'll track the following information about `users`:

```json
{
    id: 'unique ID',
    first_name: 'first name',
    last_name: 'last name',
    email: 'email address',
    role: 'student or professor'
}
```

* GET `/users`: list all users
* GET `/users/{:id}`: retrieve a user detail by ID
* POST `/users`: create a user
* PATCH `/users/{:id}`: update a user indexed by ID
* DELETE:

## Courses

This resource manages courses. It will have the following attributes:

```json
{
    id: 'unique ID of this course',
    professor_id: 'id of professor teaching this course',
    name: 'course name',
    year: 'course year',
    semester: 'fall, spring or summer',
}
```

* GET `/courses`: list all courses
* GET `/courses?proffessor={:professor_id}`: all courses offered by a given professor
* GET `/courses?student={:student_id}`: all courses taken by a student
* POST `/courses`: create a course. Details of the course (`name`, `professor_id`, `semester` will be passed as a JSON object in the POST body)

## Course Students

This resource manages students taking a course.

It will have the following attributes:

```json
{
    course_id: 'unique ID of the course',
    student_id: 'ID of student',
    grade: 'the current grade of the student, might be computed'
}
```

* GET `/course_students?course_id={:id}`: list students for a given `course_id`
* POST `/course_students/course_id={:id}`: add a student to a course. The body of the POST will specify the student by their ID.
* GET `/course_students?course_id={:id}/{:student_id}` retrieve information the students status in this course, including their course grade.
* DELETE `/course_students/course_id={:id}`: delete student from a course

## Assignments

An `assignment` will have the following attributes:

```json
{
    id: 'unique assignment ID',
    course_id: 'the course to which this assignment belongs',
    name: 'name of the assignment',
    due_date: 'due date',
    total_points: 'total points for this assignment'
}
```
* GET `/assignments/course_id={:course_id}`: list assignments by course
* POST `/assignments/course_id={:course_id}`: create an assignment
* GET `/assignments/{:id}`: retrieve assignment details by it's ID
* PATCH `/assignments/{:id}`: update an assignment.
* DELETE `/assignments/{:id}`: delete the assignment

## Assignment Grades

Assignment Grades links assignments with students
```json
{
    assignment_id: 'assignment this grade belongs to',
    user_id: 'the student being assigned a grade',
    points: 'number of points allocated for this assignment'
    grade: 'pick your poison'
}
```
* GET `/assignment_grades?assignent_id={:id}`
* POST `/assignment_grades?assignment_id={:id}`: set a student's grade for an assignment. The body will contain the `user_id` and `grade`. Note that this operation should prevent students who aren't enrolled in a course from being assigned a grade. This kind of logic would be enforced in the model layer described above.
* PATCH `assignment_grades?assignment_id={:id}`: update a student's grade for an assignment. The body of the operation will contain an object that references the `student_id` and their `grade`.


# Lab/Homework: A Flask API for a User Resource

For this lab, we'll have to leave the confines of the Jupyter notebook to run the app server. The API will be implemented with the Flask framework. Source code for a skeleton starter project is [here](https://www.dropbox.com/s/ud7czf8ejeyem2u/university-api-server.zip?dl=0). Download and unzip the file. This will create a folder called `university-api-server`.

`cd` into the folder and start up flask like so:

> ```bash
$ export FLASK_APP=api.py
$ export FLASK_ENV=development
$ flask run

By default this will start the server on port `5000`.

You can try accessing the server by navigating to `http://localhost:5000/users` on your browser.

I have taken the liberty of stubbing out the operations to list, create, read, and update `users` in. Your job in this exercise is understand the structure of a flask app and fill in the implementation of each stubbed operation. Note that in this iteration of the app, the `user` resource isn't persisted yet. Instead, you will store updates to `users` in a Python array. This means that all changes will be lost whenever the server is taken down. We'll be extending this app with a database layer in the coming weeks.

To simplify your task, I've also created a set of unit tests below to ensure that your API adheres to some basic requirements. Study the the unit test code closely. It shows basic usage of `requests`, a Python library for HTTP clients. I highly recommend your looking at the documentation [here](http://docs.python-requests.org/en/master/).

## Imports



In [19]:
import unittest
import requests
import json

# The base URL for all HTTP requests
BASE = 'http://localhost:5000/users'

# set Content-Type to application/json for all HTTP requests
headers={'Content-Type': 'application/json'}

## Problem Set
*60 Points Total*

You'll be implementing the following operations on a `/users` resource:

| METHOD                       | Description                   |
| ---------------------------- | ----------------------------- |
| `GET /users`                 | List all users                |
| `POST /users`                | Create a User                 |
| `GET /users/:id`             | Retrieve a user by `id`       |
| `PUT/PATCH /users/:id`       | Update a user with given `id` |
| `POST /users/:id/deactivate` | Deactivate a user             |

Note that we won't support `DELETE` on this resource. We'll want to prevent users from being deleted. This is because they are typically retained for historical purposes. For this reason, you implement a `/deactivate` operation instead.

## Problem 1: List Users
*10 Points*

Modify `api.py` to retrieve the collection of users. Essentially, you will return the contents of the `USERS` as JSON. See the documentaton of [`jsonify`](http://flask.pocoo.org/docs/1.0/api/) for details on how to convert a Python object to JSON.

Run the test below to show that your code is correct.


In [20]:

class Problem1Test(unittest.TestCase):
    
    # test
    def test_users_get_collection(self):
        r = requests.get(BASE, headers = headers)
        self.assertEqual(r.status_code, 200)
        
        j = r.json()
        self.assertEqual(type(j), list)
        self.assertGreater(len(j), 0)
        
        # extract the first element of the list
        first = j[0]

        # check all attributes exist
        self.assertIn('id', first)
        self.assertIn('first', first)
        self.assertIn('last', first)
        self.assertIn('email', first)
        self.assertIn('role', first)
        self.assertIn('active', first)

    

# Run the unit tests          
unittest.main(defaultTest="Problem1Test", argv=['ignored', '-v'], exit=False)
                         

test_users_get_collection (__main__.Problem1Test) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.007s

OK


## Problem 2: Retrieve a Single User
*10 Points*

Add a method to retrieve a single user by ID. That is create a function that will route to

>```bash
GET /users/<id>
```
    
See the Flask documentation for [Routing](http://flask.pocoo.org/docs/1.0/quickstart/#routing) for details on how to bind a parameter to function argument.

This method shall return an HTTP status code of `200` on success and `404` (not found) if the user with the specified ID does not exist. See the unit tests below.

In [21]:

class Problem2Test(unittest.TestCase):
    
    def test_users_get_member(self):
        
        r = requests.get(BASE + '/0')
        self.assertEqual(r.status_code, 200)
        print(r.headers)
        j = r.json()
        
        self.assertIs(type(j), dict)
        self.assertEqual(j['id'], 0)
        self.assertIn('first', j)
        self.assertIn('last', j)
        self.assertIn('email', j)
        self.assertIn('role', j)
        
    def test_users_wont_get_nonexistent_member(self):
        
        r = requests.get(BASE + '/1000')
        self.assertEqual(r.status_code, 404)
    
# Run the unit tests          
unittest.main(defaultTest="Problem2Test", argv=['ignored', '-v'], exit=False)
                         

test_users_get_member (__main__.Problem2Test) ... ok
test_users_wont_get_nonexistent_member (__main__.Problem2Test) ... 

{'Content-Type': 'application/json', 'Content-Length': '126', 'Server': 'Werkzeug/0.14.1 Python/3.6.5', 'Date': 'Sat, 23 Feb 2019 16:44:12 GMT'}


ok

----------------------------------------------------------------------
Ran 2 tests in 0.032s

OK


## Problem 3: Create a User
*10 Points*

Create a user with the following route:

>```bash
POST /users
```

The object to be created will be passed as JSON in the HTTP body. The unit test below shows how. It will be of the form:

>```json
{
    'first': 'first name',
    'last': 'last name',
    'email': 'email address',
    'role': 'professor or student',
}
```

Use `request.get_json()` to extract the body as JSON from the HTTP request.

All of these parameters are required and your code should enforce this. If validation succeeds, add the new user to the `USERS` list and give it a unique ID. 

Return HTTP status code `201` (created) if the operation succeeds and `422` (Unprocessable Entity) if validation fails.

The created user will be returned as JSON if the operation succeeds.

Future versions of your app will enforce validation constraints more rigorously.

In [22]:

class Problem3Test(unittest.TestCase):
    
    
    def test_users_create(self):
        data = json.dumps({'first': 'Sammy', 'last': 'Davis', 'email': 'sammy@cuny.edu'})

        r = requests.post(BASE, headers = headers, data = data)
        self.assertEqual(r.status_code, 201)
        
    def test_wont_create_user_without_first_name(self):
        # simple validation (missing parameters)
        data = json.dumps({'last': 'Davis', 'email': 'sammy@cuny.edu'})

        r = requests.post(BASE, headers = headers, data = data)
        self.assertEqual(r.status_code, 422)
        
        
# Run the unit tests          
unittest.main(defaultTest="Problem3Test", argv=['ignored', '-v'], exit=False)
                         

test_users_create (__main__.Problem3Test) ... ok
test_wont_create_user_without_first_name (__main__.Problem3Test) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.026s

OK


## Problem 4: Update a User
*10 Points*

Update a user with the following route:

>```bash
PATCH/PUT /users/<id>
```
    
The parameters will be passed in the HTTP body and will be an object with a subset of the user attributes.
    
Return status code `200` on success, `404` if the user was not found, and `422` if another error occurred.

In [23]:

class Problem4Test(unittest.TestCase):
        
    def test_users_update_member(self):
        data = json.dumps({'first': 'testing'})
        r = requests.patch(BASE + '/0', headers = headers, data = data)
        self.assertEqual(r.status_code, 200)
        
        j = r.json()
        self.assertIs(type(j), dict)
        self.assertEqual(j['id'], 0)
        self.assertEqual(j['first'], 'testing')
        
        # now retrieve the same object to ensure that it was really updated
        r = requests.get(BASE + '/0', headers = headers, data = data)
        self.assertEqual(r.status_code, 200)
        
        j = r.json()
        self.assertEqual(j['first'], 'testing')
        
        
    def test_users_update_member_not_found(self):
        data = json.dumps({'first': 'testing'})
        r = requests.patch(BASE + '/1000', headers = headers, data = data)
        self.assertEqual(r.status_code, 404)
        
        

# Run the unit tests          
unittest.main(defaultTest="Problem4Test", argv=['ignored', '-v'], exit=False)
                         

test_users_update_member (__main__.Problem4Test) ... ok
test_users_update_member_not_found (__main__.Problem4Test) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.028s

OK


## Problem 5: Deactivate a User

Deactivate a user with the route

>```bash
POST /users/<id>/deactivate
```
    
This method will essentially toggle the `active` attribute for the user. Return `200` on success.
    
This problem shows how to implement non-REST commands.

In [24]:
class Problem5Test(unittest.TestCase):
        
    def test_users_deactivate_member(self):

        r = requests.post(BASE + '/0/deactivate', headers = headers)
        self.assertEqual(r.status_code, 200)
        
        j = r.json()
        self.assertIs(type(j), dict)
        self.assertEqual(j['active'], False)
        

# Run the unit tests          
unittest.main(defaultTest="Problem5Test", argv=['ignored', '-v'], exit=False)


test_users_deactivate_member (__main__.Problem5Test) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.008s

OK


## Problem 6
*10 Points*

It's time to start thinking about group projects. Please propose, in 100 words or less, a project you would like to do.

*E-Comerce web app*

I want to developed an e-comerce web application. I will have a relation database(postgress) and microservice api.
web application will always connect to api server and will any kind of operation will be done through the api. 


## api.py

In [ ]:
import os
from flask import Flask, request, Response, jsonify
from functools import wraps
import json


# Custom error handler. Raise this exception
# to return a status_code, message, and body
class InvalidUsage(Exception):
    status_code = 400

    def __init__(self, message, status_code=None, payload=None):
        Exception.__init__(self)
        self.message = message
        if status_code is not None:
            self.status_code = status_code
        self.payload = payload

    def to_dict(self):
        rv = dict(self.payload or ())
        rv['message'] = self.message
        return rv


print(__name__)

app = Flask(__name__)

# set the default error handler
@app.errorhandler(InvalidUsage)
def handle_invalid_usage(error):
    response = jsonify(error.to_dict())
    response.status_code = error.status_code
    return response


if __name__ == '__main__':
    app.run(debug=True)

# dummy users
USERS = [
    {'id': 0, 'first': 'Joe', 'last': 'Bloggs',
        'email': 'joe@bloggs.com', 'role': 'student', 'active': True},
    {'id': 1, 'first': 'Ben', 'last': 'Bitdiddle',
        'email': 'ben@cuny.edu', 'role': 'student', 'active': True},
    {'id': 2, 'first': 'Alissa P', 'last': 'Hacker',
        'email': 'missalissa@cuny.edu', 'role': 'professor', 'active': True},
]

# Your code here...
# E.g.,
# @app.route("/users", methods=["GET"])

# Problem 1
@app.route("/users", methods=["GET"])
def get_users():
    return jsonify(USERS)


# Problem 2
@app.route("/users/<int:id>", methods=["GET"])
def get_user(id):
    
    selected_user = None
    for user in USERS:
        if user['id'] == id:
            selected_user = user
            break
    if selected_user == None:
        raise InvalidUsage("user not found" , 404)
    else:
        return jsonify(selected_user)

# Problem 3
@app.route("/users", methods=["POST"])
def create_user():
    if request.headers['Content-Type'] == 'application/json':
        pdata = request.get_json()
        first = pdata.get('first')
        last = pdata.get('last')
        email = pdata.get('email')
        
        if first == None or last == None or email == None:
            raise InvalidUsage("nprocessable Entity", 422)
            
        
        # defult role is student
        if pdata.get('role') is None:
            pdata['role'] = 'Student'
            
        if pdata.get('active') is None:
            pdata['active'] = False
            
        maxId =  max(USERS,key=lambda item:item['id'])['id']
      
        pdata['id'] = maxId+1;
        USERS.append(pdata);
        
        response = jsonify(pdata)
        response.status_code = 201
        return response
    
    else:
        raise InvalidUsage("Unsupported Media Type", 415)


# Problem 4

@app.route("/users/<int:id>", methods=["PATCH", "PUT"])
def updated_user(id):
    if request.headers['Content-Type'] == 'application/json':
        selected_user = None
        for user in USERS:
            if user['id'] == id:
                selected_user = user
                break
        if selected_user == None:
            raise InvalidUsage("user not found" , 404)
            
        pdata = request.get_json() 
        if pdata.get('first') != None:
            selected_user['first'] = pdata.get('first')
        if pdata.get('last') != None:
            selected_user['last'] = pdata.get('last') 
        if pdata.get('email') != None:
            selected_user['email'] = pdata.get('email') 
        if pdata.get('role') != None:
            selected_user['role'] = pdata.get('role')     
        if pdata.get('active') != None:
            selected_user['active'] = pdata.get('active')   
        
        response = jsonify(selected_user)
        response.status_code = 200
        return response
    else:
        raise InvalidUsage("Unsupported Media Type", 415)
        

# Problem 5
@app.route("/users/<int:id>/deactivate", methods=["POST"])
def delete_user(id):
    selected_user = None
    for user in USERS:
        if user['id'] == id:
            selected_user = user
            break
        if selected_user == None:
            raise InvalidUsage("user not found" , 404)
            
    selected_user['active'] = False;
    
    response = jsonify(selected_user)
    response.status_code = 200
    return response